In [1]:
import numpy as np
import random
import time
from itertools import combinations

In [2]:
def compute_subset_bandwidth(subset):
    BANDWIDTH = 10
    n = len(subset)
    if (n==1):
        return  n*BANDWIDTH/2
    bandwidth = n*(n-1)*BANDWIDTH/2 + n*BANDWIDTH/2
    return bandwidth

In [3]:
def check_camera_coverage(solution,subsets,set_all_cameras):
    set_ = []
    
    for s in solution:
        sub = subsets[s]
        for c in sub:
            set_.append(c)
    set_ = set(set_)
    return set_==set(set_all_cameras)
#check_camera_coverage([[1,1,1],[2,2,2]],{0,1,2})

In [4]:
def compute_subset_accuracy(subset,set_all_accs):
    ## Will implement later
    n = len(subset)
    sum_ = 0
    for s in subset:
        sum_ += set_all_accs[s]
    sum_ = sum_/n
    return sum_

In [5]:
def process_data(filepath):
    
    file_ = open(filepath, "r")
    lines = file_.readlines() 
    set_all_accs =  [int(x.rstrip()) for x in lines]
    total_bandwidth = set_all_accs[0]
    set_all_accs = set_all_accs[1:]
    set_all_cameras = [i for i in range(len(set_all_accs))]
    subsets = []
    subsets_bandwidth = []
    subsets_acc = []
    for i in range(1,5):
        combs = combinations(range(len(set_all_cameras)), i)
        for subset in combs:
            subsets.append(subset)
            subsets_bandwidth.append(compute_subset_bandwidth(subset))
            subsets_acc.append(compute_subset_accuracy(subset,set_all_accs))
            

    total_num_subsets = [i for i in range(len(subsets))]
    print(subsets)
    
    ''' We need to set out how the inputs will be.
    First, how would we set out our subsets in the dataset. 
    Will it be randomly sized sets (perhaps based on some other factors like geography)?
    Or will it just be pairs or triplets, each with a total bandwidth cost as a constraint?

    Second, should the accuracy cost which we are max be associated with each subset, or computed dynamically for each subset?
    But even if computed dynamically, we still need an accuracy associated with each camera element or subset to do the exponentially decaying number
    I've tried to do that below.
    '''

    ''' This should return a list or array of subsets; 
    and another 1 or 2 lists or arrays of associated costs/constraints; or
    use a dictionary where the keys are the subsets with bandwidth and accuracy keys as the associated values'''

    ''' Here I've assumed that this takes the data file, and returns the total number of subsets, a set of all cameras, the list of subsets, the accuracy of the subsets and the bandwidth'''
    return total_bandwidth,total_num_subsets, set_all_cameras, subsets, subsets_acc, subsets_bandwidth


'''
100

1,2,3,4,5...100

Subset Bandwidth
1,2,3  100
2,3,4  200


Camera Acc
1       0.5
2       0.7
....

'''

total_bandwidth,total_num_subsets, set_all_cameras, subsets, subsets_acc, subsets_bandwidth = process_data("input.txt")

[(0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (5, 6), (5, 7), (5, 8), (5, 9), (6, 7), (6, 8), (6, 9), (7, 8), (7, 9), (8, 9), (0, 1, 2), (0, 1, 3), (0, 1, 4), (0, 1, 5), (0, 1, 6), (0, 1, 7), (0, 1, 8), (0, 1, 9), (0, 2, 3), (0, 2, 4), (0, 2, 5), (0, 2, 6), (0, 2, 7), (0, 2, 8), (0, 2, 9), (0, 3, 4), (0, 3, 5), (0, 3, 6), (0, 3, 7), (0, 3, 8), (0, 3, 9), (0, 4, 5), (0, 4, 6), (0, 4, 7), (0, 4, 8), (0, 4, 9), (0, 5, 6), (0, 5, 7), (0, 5, 8), (0, 5, 9), (0, 6, 7), (0, 6, 8), (0, 6, 9), (0, 7, 8), (0, 7, 9), (0, 8, 9), (1, 2, 3), (1, 2, 4), (1, 2, 5), (1, 2, 6), (1, 2, 7), (1, 2, 8), (1, 2, 9), (1, 3, 4), (1, 3, 5), (1, 3, 6), (1, 3, 7), (1, 3, 8), (1, 3, 9), (1, 4, 5), (1, 4, 6), (1, 4, 7), (1, 4, 

In [6]:
def greedy_randomized_construction(alpha, total_num_subsets, set_all_cameras):

    # set of subsets in solution
    MAX = 1000000
    solution = set()
    #solution_camera_attributes = set() # this is actually a set of subsets or set of cameras
    solution_camera_attributes = []
    candidates = set(total_num_subsets) # this is actually a set of all the subset that we have by index

    # we need to determine the range of the RCL with the alpha and some metric
    # for example, we can use the ratio of accuracy/bandwidth which is what I have done here
    # or we can just directly use accuracy
    # this controls the range of candidates that can go into the RCL
    # While we have not covered all cameras, run the parts below
    while not check_camera_coverage(solution, subsets,set_all_cameras):
        # add to RCL while not all cameras not covered yet
        ratio = dict()

        # Calculate the ratio for every subset in the candidate set
        # Basically we run through every subset possible (i.e. each subset is a candidate)
        # See if this subset is already in the solution_camera_attributes
        # if it is, then we compute a ratio
        # This ratio is used to determine the GRASP alpha
        for i in candidates:
            # basically we want to check through whether the subset has the camera or not
            #attribute_to_add = len(subsets[i] - solution_camera_attributes)
            #print(attributes_to_add)

            #if attribute_to_add > 0:
            if i not in solution:
                # we need to see whether this greedy metric works
                ratio[i] = subsets_acc[i] / subsets_bandwidth[i]
                #print(ratio[i])
                # ratio[i] = subsets_acc[i] /total accuracy

        c_min = min(ratio.values())
        c_max = max(ratio.values())
        
        RCL = [i for i in candidates if i in ratio.keys() and ratio[i] <= c_min + alpha * (c_max - c_min)]
        #print(len(candidates),len(RCL))
        selected_index = random.choice(RCL)

        # take out that specific subset's index from the set of candidates (subset)
        candidates -= {selected_index}
        # add the index of the subset to the solution
        solution.add(selected_index)
        # add the subset to the solution_camera_attributes list
        solution_camera_attributes.append(subsets[selected_index]) 
    return solution
        
solution = greedy_randomized_construction(0.1, total_num_subsets, set_all_cameras)



In [7]:
def compute_accuracy(solution):
    sum_ = 0
    for s in solution:
        sum_+=subsets_acc[s]
    return sum_
        

In [8]:
def compute_bandwidth(solution):
    sum_ = 0
    for s in solution:
        sum_+=subsets_bandwidth[s]
    return sum_

In [9]:
def is_feasible(solution,total_bandwidth):
    return check_camera_coverage(solution,subsets,set_all_cameras) and compute_bandwidth(solution) <= total_bandwidth

In [10]:
def local_search(solution):
    # we take the current solution and the unused subsets
    # iterate and swap if it leads to a better solution in terms of higher accuracy

    # Subsets not in solution
    subsets_not_in_solution = set([s for s in total_num_subsets if s not in solution])
    #print(solution)
    best_accuracy = compute_accuracy(solution)
    #print(best_accuracy)
    best_solution_set = solution.copy()

    temp_solution = solution.copy()
    for s_out in solution:
        for s_in in subsets_not_in_solution:
            # recall we want higher accuracy so we swap in s_in if it has higher acc
            if subsets_acc[s_in] > subsets_acc[s_out]:
                # do the swap if we can get better accuracy based on simple comparison of the 2 subsets accuracy
                temp_solution.update({s_in})
                temp_solution.difference_update({s_out})

                accuracy = compute_accuracy(temp_solution)

                if accuracy > best_accuracy and is_feasible(temp_solution,total_bandwidth):
                    # update the current best accuracy and solution
                    best_accuracy = accuracy
                    best_solution_set = temp_solution
                    #print(best_accuracy,compute_bandwidth(temp_solution))
    return best_solution_set
local_search(solution)

{0,
 1,
 2,
 3,
 5,
 7,
 10,
 11,
 12,
 14,
 16,
 18,
 19,
 20,
 21,
 22,
 24,
 25,
 26,
 27,
 28,
 29,
 31,
 32,
 33,
 35,
 37,
 39,
 40,
 45,
 46,
 47,
 48,
 55,
 56,
 57,
 58,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 71,
 73,
 75,
 76,
 81,
 82,
 83,
 84,
 89,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 101,
 102,
 103,
 104,
 106,
 108,
 109,
 110,
 111,
 112,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 137,
 138,
 139,
 140,
 145,
 146,
 147,
 148,
 153,
 156,
 157,
 158,
 165,
 167,
 168,
 169,
 170,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 200,
 201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 211,
 212,
 214,
 215,
 216,
 217,
 218,
 221,
 222,
 223,
 224,
 226,
 229,
 230,
 231,
 232,
 236,
 237,
 239,
 240,
 241,
 242,
 249,
 250,
 251,
 252,
 253,
 254,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 266,
 267,
 26

In [11]:
def repair_unfeasible(solution):
    subsets_not_in_solution = set([s for s in total_num_subsets if s not in solution])
    best_bandwidth = compute_bandwidth(solution)

    #best_bandwidth = compute_bandwidth(solution)
    #best_solution_set = solution.copy()

    temp_solution = solution.copy()

    for s_out in solution:
        subs = subsets[s_out]
        temp_solution.difference_update({s_out})
        if len(subs)>1:
            for s_in in subs: 
                temp_solution.update({s_in})
            #if bandwidth < best_bandwidth:

            if is_feasible(temp_solution,total_bandwidth):
                return temp_solution
            #solution = temp_solution
    return None

In [12]:
def GRASP(num_iterations, alpha, num_sample_pool, path_relinking_flag=True):
    best_acc = 0
    best_sol = None
    # list of solutions that we have gone through
    S = []
    iteration = num_iterations
    while iteration > 0:
        iteration -= 1
        solution = greedy_randomized_construction(alpha, total_num_subsets,set_all_cameras)
        solution = local_search(solution)
        #print("F",solution)
        if not is_feasible(solution,total_bandwidth):
            solution = repair_unfeasible(solution)
            #print("D",solution)
        S.append(solution)
        #print("Best Accuracy: ",best_acc)
        acc = compute_accuracy(S[-1])
        if acc > best_acc:
            best_acc = acc
            best_sol = S[-1]
        print("Best Accuracy: ",best_acc, " Bandwidth: ",compute_bandwidth(best_sol))
    return best_acc, best_sol

best_acc,best_sol = GRASP(100, 0.1, 10, False)
print(best_acc,compute_bandwidth(best_sol))

Best Accuracy:  1570.75  Bandwidth:  930.0
Best Accuracy:  1591.0  Bandwidth:  930.0
Best Accuracy:  1597.75  Bandwidth:  930.0
Best Accuracy:  1597.75  Bandwidth:  930.0
Best Accuracy:  1597.75  Bandwidth:  930.0
Best Accuracy:  1597.75  Bandwidth:  930.0
Best Accuracy:  1597.75  Bandwidth:  930.0
Best Accuracy:  1601.75  Bandwidth:  930.0
Best Accuracy:  1601.75  Bandwidth:  930.0
Best Accuracy:  1601.75  Bandwidth:  930.0
Best Accuracy:  1601.75  Bandwidth:  930.0
Best Accuracy:  1601.75  Bandwidth:  930.0
Best Accuracy:  1601.75  Bandwidth:  930.0
Best Accuracy:  1601.75  Bandwidth:  930.0
Best Accuracy:  1601.75  Bandwidth:  930.0
Best Accuracy:  1601.75  Bandwidth:  930.0
Best Accuracy:  1606.25  Bandwidth:  930.0
Best Accuracy:  1606.25  Bandwidth:  930.0
Best Accuracy:  1623.0  Bandwidth:  930.0
Best Accuracy:  1623.0  Bandwidth:  930.0
Best Accuracy:  1623.0  Bandwidth:  930.0
Best Accuracy:  1623.0  Bandwidth:  930.0
Best Accuracy:  1623.0  Bandwidth:  930.0
Best Accuracy:  1